In [ ]:
from manim import *
import sympy as smp
import numpy as np
import scipy as sp
from itertools import product
import joblib as jl
from tqdm import tqdm
import pickle
import math

In [ ]:
with open("./store/results.pkl", "rb") as f:
    results = pickle.load(f)

In [ ]:
c = [i for i in np.linspace(0, int(math.sqrt(len(results))), 10, dtype=int)]

anss = [
    [0 for _ in range(int(math.sqrt(len(results))))]
    for _ in range(int(math.sqrt(len(results))))
]

for i,j, ans in results:
    anss[i][j] = ans

nans = [[0 for i in range(10)] for j in range(10)]

for i in range(0,50,5):
    for j in range(0,50,5):
        nans[i//5][j//5] = anss[i][j]


In [ ]:

def get_x1_y1_x2_y2( theta1, theta2, l1, l2):
    x1 = l1 * np.sin(theta1)
    y1 = -l1 * np.cos(theta1)
    x2 = x1 + l2 * np.sin(theta2)
    y2 = y1 - l2 * np.cos(theta2)
    return x1, y1, x2, y2


In [ ]:
%%manim -ql -v CRITICAL DoublePendulumScene2

t = np.linspace(0, 30, 1000)

class DoublePendulumScene2(MovingCameraScene):
    def construct(self):
        self.camera.background_color = BLACK

        r = len(nans)
        c = len(nans[0])

        w = self.camera.frame.get_width()
        h = self.camera.frame.get_height()

        infos = []

        for i in range(c):
            for j in range(r):
                
                x1, y1, x2, y2 = get_x1_y1_x2_y2(
                    nans[i][j].T[0], nans[i][j].T[2], 1, 1
                )
                color=RED
                # if i==0 and j==0:
                #     print(x1[0],y1[0],x2[0],y2[0]);
                #     color=YELLOW
                    
                origin = (i * w / r - w / 2 + w / (2 * r)) * RIGHT + (
                    j * h / c - h / 2 + h / (2 * c)
                ) * DOWN
                
                scale = 8 / (c*4)

                nx1, ny1, nx2, ny2 = x1 * scale, y1 * scale, x2 * scale, y2 * scale

                rod1 = Line(origin, origin + [nx1[0], ny1[0], 0], color=color)
                rod2 = Line(
                    origin + [nx1[0], ny1[0], 0],
                    origin + [nx2[0], ny2[0], 0],
                    color=BLUE,
                )

                infos.append(
                    {
                        "rod1": rod1,
                        "rod2": rod2,
                        "data": {"x1": nx1, "y1": ny1, "x2": nx2, "y2": ny2},
                        "origin": origin,
                    }
                )

        self.add(*(info["rod1"] for info in infos), *(info["rod2"] for info in infos))

        def update_pendulum(mob, dt):
            frame_idx = int(self.renderer.time * len(t[t<1]))
            if frame_idx >= len(t):
                frame_idx = len(t) - 1

            for info in infos:
                origin = info["origin"]

                x1 = info["data"]["x1"]
                y1 = info["data"]["y1"]
                x2 = info["data"]["x2"]
                y2 = info["data"]["y2"]

                rod1 = info["rod1"]
                rod2 = info["rod2"]

                rod1.put_start_and_end_on(
                    origin, origin + [x1[frame_idx], y1[frame_idx], 0]
                )
                rod2.put_start_and_end_on(
                    origin + [x1[frame_idx], y1[frame_idx], 0],
                    origin + [x2[frame_idx], y2[frame_idx], 0],
                )

        for m in [*(info["rod1"] for info in infos), *(info["rod2"] for info in infos)]:
            m.add_updater(update_pendulum)

        self.wait(t[-1])